In [1]:
# Importing model
from pyspark.ml.recommendation import ALS
# Importing evaluator
from pyspark.ml.evaluation import RegressionEvaluator

In [2]:
# Reading data
df = spark.read.csv('/FileStore/tables/movielens_ratings.csv',
              inferSchema=True,
              header=True)

In [3]:
# Showing some data
df.show(5)

+-------+------+------+
movieId|rating|userId|
+-------+------+------+
 2| 3.0| 0|
 3| 1.0| 0|
 5| 2.0| 0|
 9| 4.0| 0|
 11| 1.0| 0|
+-------+------+------+
only showing top 5 rows

In [4]:
# Splitting data into train and test sets
(train_set, test_set) = df.randomSplit([0.8, 0.2])

In [5]:
# Creating model
als = ALS(maxIter=5, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating').fit(train_set)

In [6]:
# Predicting results on test dataset
predictions = als.transform(test_set)

In [7]:
# Showing some results
predictions.show(10)

+-------+------+------+----------+
movieId|rating|userId|prediction|
+-------+------+------+----------+
 31| 3.0| 7| 1.9905949|
 31| 1.0| 0| 3.0071664|
 85| 1.0| 26| 6.3535085|
 85| 1.0| 5| 1.2759376|
 85| 1.0| 4| 2.1072118|
 85| 5.0| 8| 3.00011|
 85| 1.0| 25| 2.4613934|
 85| 1.0| 2| 0.528729|
 65| 1.0| 22| 0.5029093|
 53| 3.0| 20|0.38142297|
+-------+------+------+----------+
only showing top 10 rows

In [8]:
# Creating evaluator 
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

In [9]:
# Evaluating predictions
rmse = evaluator.evaluate(predictions)

In [10]:
print('RMSE: {}'.format(rmse))

RMSE: 1.90344355189

In [11]:
# Filtering a single user
single_user = test_set.filter(test_set['userId']==11).select(['movieId', 'userId'])

In [12]:
# Showing movies this particular user rated
single_user.show(5)

+-------+------+
movieId|userId|
+-------+------+
 10| 11|
 11| 11|
 23| 11|
 25| 11|
 38| 11|
+-------+------+
only showing top 5 rows

In [13]:
# Recomending other movies (prediction)
recommendations = als.transform(single_user)

In [14]:
# Showing some results
recommendations.orderBy('prediction', ascending=False).show()

# The basic ideia is how likely is this user going to like movie titles based on another user's ratings and similar items. So, for example, it is likely that this user is going to like movieId 23. On the other hand, this same user probably won't like movieId 51.

+-------+------+----------+
movieId|userId|prediction|
+-------+------+----------+
 10| 11| 4.508434|
 23| 11| 3.4031239|
 11| 11| 2.2950306|
 38| 11| 2.097754|
 25| 11|0.24203281|
 69| 11|-1.0637807|
 51| 11|-2.5631297|
+-------+------+----------+